# 感情分析 (Sentiment Analysis)

文章の特定の単語などから、文章の感情がポジティブかネガティブかを判別

### ソフトウェア

以下のような簡単に使用できるものもあるが、依存関係が面倒なため、ここでは辞書を使用
- oseti https://github.com/ikegami-yukino/oseti

### 辞書の取得

単語感情極性対応表
http://www.lr.pi.titech.ac.jp/~takamura/pndic_ja.html
（文字化けする場合は 日本語 (EUC) に設定）
- 日本語 http://www.lr.pi.titech.ac.jp/~takamura/pubs/pn_ja.dic<br>
  再配布は禁止のため、各自 data/pn_ja.dic として保存
- 単語ごとの評価極性のスコア [-1, 1]

In [ ]:
import pandas as pd
import numpy as np
import spacy

# 日本語モデル
nlp = spacy.load('ja_core_news_lg')

# フィードデータの読み込み、確認
feeds = pd.read_csv('data/output_jp.csv')

# title と summary を結合
# str.cat() により複数列の文字列を結合
# - sep=' ': 間に挟む文字列
# - na_rep='': NaN は空文字列に変換（指定しないと結合結果が NaN になる）
feeds['text'] = feeds['title'].str.cat(feeds['summary'], sep='。', na_rep='')

# 不要になった列を削除した処理用の DataFrame
df = feeds.drop(['title', 'summary'], axis=1)

# 確認
df.head()

### 日本語テキストに対する前処理

In [ ]:
# 不要な単語を除去
# - ストップワード (is_stop)
# - いくつかの品詞
#     AUX: 助動詞
#     PUNCT: 句読点
#     SPACE: 空白文字
#     SYM: 記号
#     X: その他
# - うまく取り除けない単語や文字
stop_pos = ['AUX', 'PUNCT', 'SPACE', 'SYM', 'X']
stop_words = ['.']

def token_to_add(w):
    t = w.text    # 単語
    p = w.pos_    # 品詞
    l = w.lemma_  # 原型

    # ストップワードは None を返す
    if w.is_stop:
        return None
    if p in stop_pos:
        return None
    if l in stop_words:
        return None

    if len(l) == 0:
        return t
    return l

def preprocess(text):
    tokens = []
    
    for w in nlp(text):
        t = token_to_add(w)
        if t is not None:
            tokens.append(t)

    # トークンのリストを返す
    return tokens

### 感情極性辞書による感情分析

- 文に含まれる単語の評価極性のスコア [-1, 1] の平均値を、文のスコアとする

In [ ]:
# 例として最初の行のテキストだけを処理
text = df['text'].iloc[0]

print(preprocess(text))

In [ ]:
# 辞書の読み込み
# - ep=':': 区切り文字は ':'
# - header=None: ヘッダ無しの指定
pn_dict = pd.read_csv('data/pn_ja.dic', sep=':', encoding='shift_jis', header=None)

# 列ラベルの指定
pn_dict.columns = ['word', 'kana', 'pos', 'pn']

# 確認
pn_dict

In [ ]:
# 変数に入れた単語のスコアを確認
w = '優れる'
pn_dict[pn_dict.word == w].pn

In [ ]:
# 文に含まれる単語のスコアの平均値を計算

# 単語ごとのスコア
pn_list = []

# 各単語についてスコアを取得し pn_list に追加
for w in preprocess(text):
    pn = pn_dict[pn_dict.word == w]
    # 単語が辞書に無い場合はスキップ
    if pn.empty:
        continue
    w = pn.iloc[0]
    # 単語ごとのスコアを確認
    print(w.word, w.pn)
    pn_list.append(w.pn)

# 平均値を計算
np.average(pn_list)